In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
status = pd.read_csv('data/churn_status_preprocessing.csv')
status

,고객ID,분기,고객만족도점수,현재고객상태,이탈여부,이탈위험점수,고객생애가치,이탈유형,이탈사유
0,8779-QRDMV,Q3,3,Churned,True,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,Q3,3,Churned,True,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,Q3,2,Churned,True,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,Q3,2,Churned,True,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,Q3,2,Churned,True,67,2793,Price,Extra data charges
...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Q3,5,Stayed,False,45,5306,NaN,NaN
7039,6840-RESVB,Q3,3,Stayed,False,59,2140,NaN,NaN
7040,2234-XADUH,Q3,4,Stayed,False,71,5560,NaN,NaN
7041,4801-JZAZL,Q3,4,Stayed,False,59,2793,NaN,NaN


In [3]:
total = pd.read_csv('data/total_final.csv')
total

,고객ID,위도,경도,성별,고령자여부,배우자여부,부양가족여부,가입개월수,전화서비스가입여부,복수회선여부,...,가입혜택,장거리통화요금,월평균다운로드용량(GB),프리미엄기술지원여부,음악스트리밍이용여부,무제한데이터이용여부,총환불액,총초과데이터요금,총장거리통화요금,총납부금
0,0002-ORFBO,34.827662,-118.999073,False,False,True,False,9,True,False,...,No,42.39,16,True,False,True,0.00,0,381.51,974.81
1,0003-MKNFE,34.162515,-118.203869,True,False,False,False,9,True,True,...,No,10.69,10,False,True,False,38.33,10,96.21,610.28
2,0004-TLHLJ,33.645672,-117.922613,True,False,False,False,4,True,False,...,Offer E,33.65,30,False,False,True,0.00,0,134.60,415.45
3,0011-IGKFF,38.014457,-122.115432,True,True,True,False,13,True,False,...,Offer D,27.82,4,False,False,True,0.00,0,361.66,1599.51
4,0013-EXCHZ,34.227846,-119.079903,False,True,True,False,3,True,False,...,No,7.38,11,True,False,True,0.00,0,22.14,289.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,32.759327,-116.997260,False,False,False,False,13,True,False,...,Offer D,46.68,59,True,True,True,0.00,0,606.84,1349.74
7039,9992-RRAMN,37.734971,-120.954271,True,False,True,False,22,True,True,...,Offer D,16.20,17,False,True,True,0.00,0,356.40,2230.10
7040,9992-UJOEL,39.108252,-123.645121,True,False,False,False,2,True,False,...,Offer E,18.62,51,False,False,True,0.00,0,37.24,129.99
7041,9993-LHIEB,33.001813,-117.263628,True,False,True,False,67,True,False,...,Offer A,2.12,58,True,True,True,0.00,0,142.04,4769.69


In [4]:
## 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
## 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df):
  print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

  summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
  summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
  summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

  summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
  summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산

  summary['첫 번째 값'] = df.loc[0].values                          # 각 피처의 첫 번째 샘플 값
  summary['두 번째 값'] = df.loc[1].values                          # 각 피처의 두 번째 샘플 값
  summary['세 번째 값'] = df.loc[2].values                          # 각 피처의 세 번째 샘플 값

  return summary                                                     # 요약 테이블 반환

In [5]:
resumetable(total)

데이터셋 크기: (7043, 43)


,피처,데이터 타입,결측값 개수,고유값 개수,첫 번째 값,두 번째 값,세 번째 값
0,고객ID,object,0,7043,0002-ORFBO,0003-MKNFE,0004-TLHLJ
1,위도,float64,0,1652,34.827662,34.162515,33.645672
2,경도,float64,0,1651,-118.999073,-118.203869,-117.922613
3,성별,bool,0,2,False,True,True
4,고령자여부,bool,0,2,False,False,False
5,배우자여부,bool,0,2,True,False,False
6,부양가족여부,bool,0,2,False,False,False
7,가입개월수,int64,0,73,9,9,4
8,전화서비스가입여부,bool,0,2,True,True,True
9,복수회선여부,bool,0,2,False,True,False


In [6]:
status_selected = status[['고객ID', '이탈유형']]
total_final = pd.merge(total, status_selected, on='고객ID', how = 'outer')
resumetable(total_final)

데이터셋 크기: (7043, 44)


,피처,데이터 타입,결측값 개수,고유값 개수,첫 번째 값,두 번째 값,세 번째 값
0,고객ID,object,0,7043,0002-ORFBO,0003-MKNFE,0004-TLHLJ
1,위도,float64,0,1652,34.827662,34.162515,33.645672
2,경도,float64,0,1651,-118.999073,-118.203869,-117.922613
3,성별,bool,0,2,False,True,True
4,고령자여부,bool,0,2,False,False,False
5,배우자여부,bool,0,2,True,False,False
6,부양가족여부,bool,0,2,False,False,False
7,가입개월수,int64,0,73,9,9,4
8,전화서비스가입여부,bool,0,2,True,True,True
9,복수회선여부,bool,0,2,False,True,False


In [7]:
total_final['이탈유형'].value_counts()

이탈유형
Competitor         841
Attitude           314
Dissatisfaction    303
Price              211
Other              200
Name: count, dtype: int64

In [8]:
churntype_translation = {
    'Competitor': '경쟁사로 이탈',
    'Attitude': '태도 문제',
    'Dissatisfaction': '불만족',
    'Price': '가격 문제',
    'Other': '기타'
}

# 매핑 적용
total_final['한글이탈유형'] = total_final['이탈유형'].map(churntype_translation)

# 결과 확인
print(total_final[['이탈유형', '한글이탈유형']].head())

              이탈유형   한글이탈유형
0              NaN      NaN
1              NaN      NaN
2       Competitor  경쟁사로 이탈
3  Dissatisfaction      불만족
4  Dissatisfaction      불만족


In [9]:
reason_mapping = {
    'Competitor made better offer': '경쟁사가 더 나은 제안을 했습니다',
    'Moved': '이동했습니다',
    'Competitor had better devices': '경쟁사가 더 나은 기기를 보유했습니다',
    'Competitor offered higher download speeds': '경쟁사가 더 빠른 다운로드 속도를 제공함',
    'Competitor offered more data': '경쟁사가 더 많은 데이터를 제공함',
    'Price too high': '가격이 너무 높음',
    'Product dissatisfaction': '제품 불만족',
    'Service dissatisfaction': '서비스 불만족',
    'Lack of self-service on Website': '웹사이트의 셀프 서비스 부족',
    'Network reliability': '네트워크 안정성',
    'Limited range of services': '제한된 서비스 범위',
    'Lack of affordable download/upload speed': '저렴한 다운로드/업로드 속도 부족',
    'Long distance charges': '장거리 요금',
    'Extra data charges': '추가 데이터 요금',
    "Don't know": '모르겠다',
    'Poor expertise of online support': '온라인 지원의 전문성 부족',
    'Poor expertise of phone support': '전화 지원의 전문성 부족',
    'Attitude of service provider': '서비스 제공업체의 태도',
    'Attitude of support person': '지원 담당자의 태도',
    'Deceased': '사망'
}

In [10]:
total_final['한글이탈이유'] = total_final['이탈이유'].map(reason_mapping)
total_final.head(20)

,고객ID,위도,경도,성별,고령자여부,배우자여부,부양가족여부,가입개월수,전화서비스가입여부,복수회선여부,...,프리미엄기술지원여부,음악스트리밍이용여부,무제한데이터이용여부,총환불액,총초과데이터요금,총장거리통화요금,총납부금,이탈유형,한글이탈유형,한글이탈이유
0,0002-ORFBO,34.827662,-118.999073,False,False,True,False,9,True,False,...,True,False,True,0.00,0,381.51,974.81,NaN,NaN,NaN
1,0003-MKNFE,34.162515,-118.203869,True,False,False,False,9,True,True,...,False,True,False,38.33,10,96.21,610.28,NaN,NaN,NaN
2,0004-TLHLJ,33.645672,-117.922613,True,False,False,False,4,True,False,...,False,False,True,0.00,0,134.60,415.45,Competitor,경쟁사로 이탈,가격이 너무 높음
3,0011-IGKFF,38.014457,-122.115432,True,True,True,False,13,True,False,...,False,False,True,0.00,0,361.66,1599.51,Dissatisfaction,불만족,제품 불만족
4,0013-EXCHZ,34.227846,-119.079903,False,True,True,False,3,True,False,...,True,False,True,0.00,0,22.14,289.54,Dissatisfaction,불만족,네트워크 안정성
5,0013-MHZWF,37.581496,-119.972762,False,False,False,True,9,True,False,...,True,True,True,0.00,0,150.93,722.38,NaN,NaN,NaN
6,0013-SMEOE,34.757477,-120.550507,False,True,True,False,71,True,False,...,True,True,True,0.00,0,707.16,8611.41,NaN,NaN,NaN
7,0014-BMAQU,38.489789,-122.270110,True,False,True,False,63,True,True,...,True,False,False,0.00,20,816.48,6214.28,NaN,NaN,NaN
8,0015-UOCOJ,34.296813,-118.685703,False,True,False,False,7,True,False,...,False,False,True,0.00,0,73.71,414.06,NaN,NaN,NaN
9,0016-QLJIS,38.984756,-121.345074,False,False,True,True,65,True,True,...,True,True,True,0.00,0,1849.90,7807.80,NaN,NaN,NaN


In [11]:
# 이탈유형별 이탈이유별 개수
reason_counts_by_type = total_final.groupby('한글이탈유형')['한글이탈이유'].value_counts()

# 결과 확인
pd.DataFrame(reason_counts_by_type)

count
한글이탈유형  한글이탈이유                       
가격 문제   가격이 너무 높음                  78
        추가 데이터 요금                  40
        장거리 요금                     35
        저렴한 다운로드/업로드 속도 부족         31
        모르겠다                        6
        웹사이트의 셀프 서비스 부족             5
        제품 불만족                      5
        경쟁사가 더 나은 기기를 보유했습니다        4
        서비스 제공업체의 태도                3
        지원 담당자의 태도                  3
        경쟁사가 더 빠른 다운로드 속도를 제공함      1
경쟁사로 이탈 경쟁사가 더 빠른 다운로드 속도를 제공함    158
        경쟁사가 더 많은 데이터를 제공함        152
        경쟁사가 더 나은 제안을 했습니다        136
        경쟁사가 더 나은 기기를 보유했습니다      126
        지원 담당자의 태도                 50
        서비스 제공업체의 태도               40
        네트워크 안정성                   26
        서비스 불만족                    25
        제품 불만족                     19
        가격이 너무 높음                  18
        웹사이트의 셀프 서비스 부족            16
        추가 데이터 요금                  16
        모르겠다                       15
        장거리 요금                      9
        저렴한 다운로드/업로드 속도 부족          8
        전화 지원의 전문성 부족               8
        이동했습니다                      7
        온라인 지원의 전문성 부족              6
        제한된 서비스 범위                  6
기타      모르겠다                      133
        이동했습니다                     46
        사망                          6
        경쟁사가 더 나은 제안을 했습니다          4
        가격이 너무 높음                   2
        경쟁사가 더 빠른 다운로드 속도를 제공함      2
        경쟁사가 더 많은 데이터를 제공함          1
        네트워크 안정성                    1
        서비스 불만족                     1
        서비스 제공업체의 태도                1
        제한된 서비스 범위                  1
        지원 담당자의 태도                  1
        추가 데이터 요금                   1
불만족     제품 불만족                     77
        네트워크 안정성                   72
        서비스 불만족                    63
        제한된 서비스 범위                 37
        웹사이트의 셀프 서비스 부족            29
        온라인 지원의 전문성 부족             13
        전화 지원의 전문성 부족              12
태도 문제   지원 담당자의 태도                138
        서비스 제공업체의 태도               91
        웹사이트의 셀프 서비스 부족            38
        경쟁사가 더 빠른 다운로드 속도를 제공함     28
        경쟁사가 더 많은 데이터를 제공함          9
        저렴한 다운로드/업로드 속도 부족          5
        네트워크 안정성                    4
        제품 불만족                      1

In [12]:
total_final['한글이탈유형'].value_counts()

한글이탈유형
경쟁사로 이탈    841
태도 문제      314
불만족        303
가격 문제      211
기타         200
Name: count, dtype: int64

In [13]:
# total_final.to_csv('data/total_real_final.csv', index=False, encoding='utf-8-sig')

In [14]:
# 한글 컬럼 기준으로 필터링
filtered_df = total_final[
    (total_final['한글이탈유형'] == '가격') & 
    (total_final['한글이탈이유'] == '웹사이트의 셀프 서비스 부족')
]

# 결과 확인
print(filtered_df)


Empty DataFrame
Columns: [고객ID, 위도, 경도, 성별, 고령자여부, 배우자여부, 부양가족여부, 가입개월수, 전화서비스가입여부, 복수회선여부, 인터넷서비스유형, 온라인보안서비스여부, 온라인백업서비스여부, 기기보호서비스여부, 기술지원서비스여부, TV스트리밍이용여부, 영화스트리밍이용여부, 계약기간유형, 전자청구서이용여부, 결제방법, 월요금, 총요금, 이탈여부, 이탈여부(bool), 고객생애가치, 이탈이유, 나이, 30세미만여부, 부양가족수, 고객만족도점수, 현재고객상태, 친구추천여부, 친구추천횟수, 가입혜택, 장거리통화요금, 월평균다운로드용량(GB), 프리미엄기술지원여부, 음악스트리밍이용여부, 무제한데이터이용여부, 총환불액, 총초과데이터요금, 총장거리통화요금, 총납부금, 이탈유형, 한글이탈유형, 한글이탈이유]
Index: []

[0 rows x 46 columns]
